**Problem 1**\
Full solution for problem1

1. Data clean\
Run in the same folder with the original data.

In [ ]:
import os
import math
from collections import defaultdict

import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
income = pd.read_csv('avg_individual_income.csv')
employ = pd.read_csv('employment_rate.csv')
population = pd.read_csv('population.csv')
facility = pd.read_csv('child_care_regulated.csv')

In [3]:
def standardize_zip(df, candidates=('zipcode', 'zip_code', 'ZIP code', 'ZIP', 'Zip')):
    for c in candidates:
        if c in df.columns:
            df = df.rename(columns={c: 'zipcode'})
            break
    df['zipcode'] = (
        df['zipcode']
        .astype(str)
        .str.extract(r'(\d+)', expand=False)
        .str.zfill(5)
    )
    return df

def _zpad(x):
    return (
        pd.Series(x)
        .astype(str)
        .str.extract(r'(\d+)', expand=False)
        .str.zfill(5)
    )

In [4]:
income = standardize_zip(income)
employ = standardize_zip(employ)
population = standardize_zip(population)
facility = standardize_zip(facility)

# early-child capacity = infant + toddler + preschool
facility['early_child_capacity'] = facility[
    ['infant_capacity', 'toddler_capacity', 'preschool_capacity']
].sum(axis=1)

fac_caps = (
    facility
    .groupby('zipcode', as_index=False)
    .agg(
        total_capacity=('total_capacity', 'sum'),
        early_child_capacity=('early_child_capacity', 'sum')
    )
)

In [5]:
age_0_5_col = '-5'
age_5_9_col = '5-9'
age_10_14_col = '10-14'

population['pop_0_5'] = population[age_0_5_col]
population['pop_0_12'] = (
    population[age_0_5_col] +
    population[age_5_9_col] +
    (3/5) * population[age_10_14_col]
)

pop_sel = population[['zipcode', 'pop_0_5', 'pop_0_12']]

In [6]:
merged = (
    fac_caps
    .merge(pop_sel, on='zipcode', how='left')
    .merge(income.drop_duplicates('zipcode'), on='zipcode', how='left')
    .merge(employ.drop_duplicates('zipcode'), on='zipcode', how='left')
)

In [7]:
# Make a working df
df = merged.copy()
df['zipcode'] = _zpad(df['zipcode'])

In [8]:
# Rename capacity columns to 0_5 and 0_12
df = df.rename(
    columns={
        'early_child_capacity': '0_5_capacity',
        'total_capacity': '0_12_capacity'
    }
)

In [9]:
# Facility table capacities consistent with naming
facility = facility.rename(
    columns={
        'total_capacity': '0_12_capacity',
        'early_child_capacity': '0_5_capacity'
    }
)
# Keep only rows with population available
df = df.dropna(subset=['pop_0_5', 'pop_0_12'])

In [10]:
# High demand flag
# (column names must match your CSVs; adjust if needed)
df['high_demand'] = (
    (df['employment rate'] >= 0.6) | (df['average income'] <= 60000)
).astype(int)

In [11]:
# Indexing objects
Z_all = df['zipcode'].tolist()
zip_now_012 = df.set_index('zipcode')['0_12_capacity'].fillna(0.0).to_dict()
zip_now_05 = df.set_index('zipcode')['0_5_capacity'].fillna(0.0).to_dict()
pop_012 = df.set_index('zipcode')['pop_0_12'].to_dict()
pop_05 = df.set_index('zipcode')['pop_0_5'].to_dict()
is_hd = df.set_index('zipcode')['high_demand'].to_dict()

In [12]:
# Facility-level expansion caps: min(120% of its capacity, 500)
fac_012 = facility['0_12_capacity'].fillna(0.0)
facility['exp_cap_fac'] = np.minimum(1.2 * fac_012, 500.0)
# Zip-level total expansion cap (sum of facility caps in the zip) — not used directly as a constraint here,
# but kept if you want to audit
exp_cap_zip = facility.groupby('zipcode', as_index=False)['exp_cap_fac'].sum()
exp_cap_zip = dict(zip(exp_cap_zip['zipcode'], exp_cap_zip['exp_cap_fac']))
for z in Z_all:
    exp_cap_zip.setdefault(z, 0.0)

In [13]:
# New facility types
NEW_TYPES = {
    'S': {'tot': 100, 'u5max': 50,  'cost':  65000},
    'M': {'tot': 200, 'u5max': 100, 'cost':  95000},
    'L': {'tot': 400, 'u5max': 200, 'cost': 115000},
}
# cost per 0-5 slot
EQUIP_COST_U5 = 100.0

In [14]:
# Organize facility-level info arrays
facility = facility.copy()
facility['cap_now'] = facility['0_12_capacity'].fillna(0.0)
facility['cap_add_max'] = facility['exp_cap_fac'].fillna(0.0)
# facility match
fac_by_zip = {}
for z, grp in facility.groupby('zipcode'):
    fac_by_zip[z] = {
        'idx':  grp.index.to_list(),
        'n_now': grp['cap_now'].to_numpy(dtype=float),
        'x_max': grp['cap_add_max'].to_numpy(dtype=float),
    }
for z in Z_all:
    fac_by_zip.setdefault(z, {'idx': [], 'n_now': np.zeros(0), 'x_max': np.zeros(0)})

2. Parameter builder\
Add parameters for building and expansion.\
Including the upper bound for each zipcode.

In [15]:
def max_builds(z):
    """A small heuristic UB on new builds to tighten the model."""
    thr = 0.5 if is_hd[z] == 1 else (1.0/3.0)
    need_tot = max(thr * pop_012[z] - zip_now_012[z], 0.0)
    need_u5 = max((2.0/3.0) * pop_05[z] - zip_now_05[z], 0.0)

    need_by_tot = math.ceil(need_tot / 100.0) if need_tot > 0 else 0
    need_by_u5  = math.ceil(need_u5  /  50.0) if need_u5  > 0 else 0

    base = max(need_by_tot, need_by_u5)
    if base == 0:
        return 0, 0, 0
    ub = base + 3
    ub = int(min(ub, 30))
    return ub, ub, ub


In [16]:
# time limited detection
def _make_model(name='p1_statewide', mipgap=1e-4, time_limit=None, verbose=False):
    m = gp.Model(name)
    m.Params.MIPGap = mipgap
    if time_limit is not None:
        m.Params.TimeLimit = time_limit
    if not verbose:
        m.Params.OutputFlag = 0
    return m

In [17]:
def add_facility_expansion_vars(m, facility_df, eps=1e-6):
    """Add expansion variables per facility:
       x_fac[f] = expansion slots (continuous, 0..x_max)
       y_trig[f] = 1 if expansion >= 100% of existing capacity (cap_now), else 0
    """
    fac_idx = facility_df.index.to_list()
    fac_zip = facility_df['zipcode'].tolist()
    cap_now = facility_df['cap_now'].fillna(0.0).to_numpy(dtype=float)
    cap_add_max = facility_df['cap_add_max'].fillna(0.0).to_numpy(dtype=float)

    x_f, y_f = {}, {}
    for k, f in enumerate(fac_idx):
        x = m.addVar(lb=0.0, ub=float(cap_add_max[k]), name=f'x_fac[{f}]')
        y = m.addVar(vtype=GRB.BINARY, name=f'y_trig[{f}]')

        if cap_now[k] > 0:
            # y=1 ⇒ x >= cap_now - eps
            m.addGenConstrIndicator(y, True,  x, GRB.GREATER_EQUAL, cap_now[k] - eps, name=f'trig_on[{f}]')
            # y=0 ⇒ x <= cap_now - eps
            m.addGenConstrIndicator(y, False, x, GRB.LESS_EQUAL,   cap_now[k] - eps, name=f'trig_off[{f}]')
        else:
            # No existing capacity ⇒ cannot trigger the '≥100%' rule
            m.addConstr(y == 0, name=f'no_trig_nf0[{f}]')
        x_f[f] = x
        y_f[f] = y
    return fac_idx, fac_zip, cap_now, cap_add_max, x_f, y_f

In [18]:
def add_zip_level_vars(m, Z_all, is_hd, max_builds, NEW_TYPES):
    """Add zip-level new build counts and two U5 allocation auxiliaries."""
    nS, nM, nL, b05, e05, thr_zip = {}, {}, {}, {}, {}, {}
    for z in Z_all:
        thr_zip[z] = 0.5 if is_hd[z] == 1 else (1.0/3.0)
        ubS, ubM, ubL = max_builds(z)

        nS[z] = m.addVar(vtype=GRB.INTEGER, lb=0, ub=ubS, name=f'nS[{z}]')
        nM[z] = m.addVar(vtype=GRB.INTEGER, lb=0, ub=ubM, name=f'nM[{z}]')
        nL[z] = m.addVar(vtype=GRB.INTEGER, lb=0, ub=ubL, name=f'nL[{z}]')

        # b05 = new-build U5 slots; e05 = expansion-allocated U5 slots
        b05[z] = m.addVar(lb=0.0, name=f'b05[{z}]')
        e05[z] = m.addVar(lb=0.0, name=f'e05[{z}]')
    return nS, nM, nL, b05, e05, thr_zip

3. Constraints\
Build constraints on:
1. 0-12 threshold for different demand zipcode.
2. 0-5 threshold for each zipcode as NYS requirement
3. 0-5 threshold as it cannot exceed the 0-12 additional slot
4. Maximum expansion scale is the minimum of 1.2 scale and 500 addition slots.

In [19]:
def add_linking_constraints(m, Z_all, fac_idx, fac_zip, x_f, e05, b05, nS, nM, nL, NEW_TYPES):
    """Link e05 to expansions in the zip; bound b05 by new build U5 maxima."""
    facs_in_zip = defaultdict(list)
    for f, zf in zip(fac_idx, fac_zip):
        facs_in_zip[zf].append(f)

    # e05[z] can’t exceed total expansion in zip z
    for z in Z_all:
        sum_xz = gp.quicksum(x_f[f] for f in facs_in_zip[z]) if len(facs_in_zip[z]) > 0 else gp.LinExpr(0.0)
        m.addConstr(e05[z] <= sum_xz + 1e-9, name=f'u5_subset[{z}]')

    # b05[z] bounded by the sum of U5 caps of newly built centers
    for z in Z_all:
        m.addConstr(
            b05[z] <= NEW_TYPES['S']['u5max'] * nS[z]
                    + NEW_TYPES['M']['u5max'] * nM[z]
                    + NEW_TYPES['L']['u5max'] * nL[z],
            name=f'build05_cap[{z}]'
        )
    return facs_in_zip

In [20]:
def add_coverage_constraints(
    m, Z_all, thr_zip, pop_012, pop_05, zip_now_012, zip_now_05,
    facs_in_zip, x_f, nS, nM, nL, e05, b05, NEW_TYPES
):
    """Zip-level coverage constraints for 0–12 and under-5 populations."""
    for z in Z_all:
        tot_after = (
            zip_now_012[z]
            + gp.quicksum(x_f[f] for f in facs_in_zip[z])
            + NEW_TYPES['S']['tot'] * nS[z]
            + NEW_TYPES['M']['tot'] * nM[z]
            + NEW_TYPES['L']['tot'] * nL[z]
        )
        u5_after = zip_now_05[z] + e05[z] + b05[z]

        m.addConstr(tot_after >= thr_zip[z] * pop_012[z], name=f'cover_012[{z}]')
        m.addConstr(u5_after >= (2.0 / 3.0) * pop_05[z], name=f'cover_05[{z}]')

Objective Function

In [21]:
def set_objective(
    m, fac_idx, cap_now, x_f, y_f,
    Z_all, e05, b05, nS, nM, nL, NEW_TYPES, EQUIP_COST_U5
):
    """Objective with BOTH:
       (i) $200 per expansion slot (for all x_f) AND
       (ii) threshold cost when expansion ≥100%: $20,000 + $200 * cap_now
       plus U5 equipment and new build costs.
    """
    # Threshold (≥100%) expansion cost per facility
    expand_trigger_cost = gp.quicksum(
        (20000.0 + 200.0 * cap_now[k]) * y_f[f]
        for k, f in enumerate(fac_idx)
    )

    # Per-slot expansion cost
    additional_cost = 0.0
    expand_per_slot_cost = gp.quicksum(
        additional_cost * x_f[f] for f in fac_idx
    )

    # U5 equipment cost: $100 per new U5 slot (expansion e05 + new build b05)
    equip_cost = EQUIP_COST_U5 * (
        gp.quicksum(e05[z] for z in Z_all) +
        gp.quicksum(b05[z] for z in Z_all)
    )

    # New build cost
    build_cost = gp.quicksum(
        NEW_TYPES['S']['cost'] * nS[z] +
        NEW_TYPES['M']['cost'] * nM[z] +
        NEW_TYPES['L']['cost'] * nL[z]
        for z in Z_all
    )

    m.setObjective(
        expand_trigger_cost + expand_per_slot_cost + equip_cost + build_cost,
        GRB.MINIMIZE
    )

In [22]:
def optimize_and_collect_fac(m, status_ok_set, fac_idx, fac_zip, cap_now, cap_add_max, x_f, y_f):
    status = (
        'optimal' if m.Status == GRB.OPTIMAL
        else ('time_limit' if m.Status == GRB.TIME_LIMIT else f'status_{m.Status}')
    )
    obj = m.objVal if m.Status in status_ok_set else None

    fac_rows = []
    for k, f in enumerate(fac_idx):
        fac_rows.append({
            'facility_id': f,
            'zipcode': fac_zip[k],
            'cap_now': cap_now[k],
            'cap_add_max': cap_add_max[k],
            'expand_x': float(x_f[f].X) if obj is not None else None,
            'trigger_100pct': int(round(y_f[f].X)) if obj is not None else None
        })
    df_fac = pd.DataFrame(fac_rows)
    return status, obj, df_fac

Outcome summary function

In [23]:
def build_zip_summary(
    df_fac, Z_all, thr_zip, zip_now_012, zip_now_05,
    pop_012, pop_05, NEW_TYPES, EQUIP_COST_U5,
    nS, nM, nL, b05, e05, obj
):
    zip_rows = []
    for z in Z_all:
        pre_012, pre_05 = zip_now_012[z], zip_now_05[z]

        build_S_z = int(round(nS[z].X)) if obj is not None else None
        build_M_z = int(round(nM[z].X)) if obj is not None else None
        build_L_z = int(round(nL[z].X)) if obj is not None else None
        build_05_z = b05[z].X if obj is not None else None

        build_scale_z = (
            100 * build_S_z + 200 * build_M_z + 400 * build_L_z
        ) if obj is not None else None

        exp_012_z = df_fac.loc[df_fac['zipcode'] == z, 'expand_x'].sum() if obj is not None else None
        exp_05_z = e05[z].X if obj is not None else None

        after_012 = (pre_012 + exp_012_z + build_scale_z) if obj is not None else None
        after_05 = (pre_05 + exp_05_z + build_05_z) if obj is not None else None

        target_012 = thr_zip[z] * pop_012[z] if obj is not None else None
        target_05 = (2.0 / 3.0) * pop_05[z] if obj is not None else None

        tol = 1e-6
        meet_012 = (after_012 is not None) and (after_012 >= target_012 - tol)
        meet_05 = (after_05 is not None) and (after_05 >= target_05 - tol)
        gap_012 = max(target_012 - after_012, 0.0) if obj is not None else None
        gap_05 = max(target_05 - after_05, 0.0) if obj is not None else None

        # Threshold expansion cost (≥100% trigger)
        fac_expand_trig_cost_z = (
            df_fac.loc[df_fac['zipcode'] == z, :]
            .assign(
                trig_cost=lambda d: (20000.0 + 200.0 * d['cap_now']) * d['trigger_100pct']
            )['trig_cost'].sum() if obj is not None else None
        )

        # Per-slot expansion cost ($200 per expansion slot)
        fac_expand_slot_cost_z = (
            200.0 * df_fac.loc[df_fac['zipcode'] == z, 'expand_x'].sum()
        ) if obj is not None else None

        equip_cost_z = EQUIP_COST_U5 * (
            (exp_05_z if obj is not None else 0.0) +
            (build_05_z if obj is not None else 0.0)
        )
        build_cost_z = (
            NEW_TYPES['S']['cost'] * build_S_z +
            NEW_TYPES['M']['cost'] * build_M_z +
            NEW_TYPES['L']['cost'] * build_L_z
        ) if obj is not None else None

        obj_z = (
            fac_expand_trig_cost_z +
            fac_expand_slot_cost_z +
            equip_cost_z +
            build_cost_z
        ) if obj is not None else None

        zip_rows.append({
            'zipcode': z,
            'obj_zip': obj_z,
            'base_012': pre_012, 'base_05': pre_05,
            'exp_012_zip': exp_012_z, 'exp_05_zip': exp_05_z,
            'build_scale_zip': build_scale_z,
            'build_S': build_S_z, 'build_M': build_M_z, 'build_L': build_L_z, 'build_05': build_05_z,
            'after_012': after_012, 'after_05': after_05,
            'target_012': target_012, 'target_05': target_05,
            'meet_012': int(bool(meet_012)), 'meet_05': int(bool(meet_05)),
            'gap_012': gap_012, 'gap_05': gap_05,
            'thr_used_012': thr_zip[z]
        })

    df_zip = pd.DataFrame(zip_rows).sort_values('zipcode')
    df_builds = df_zip[['zipcode', 'build_S', 'build_M', 'build_L', 'build_05', 'build_scale_zip']].copy()
    return df_zip, df_builds

In [24]:
def write_outputs(outdir, status, obj, df_zip, df_fac, df_builds):
    os.makedirs(outdir, exist_ok=True)
    with open(os.path.join(outdir, 'overall_objective.txt'), 'w') as f:
        f.write(f"status: {status}\n")
        f.write(f"objective: {obj}\n")
    df_zip.to_csv(os.path.join(outdir, 'zip_summary.csv'), index=False)
    df_fac.to_csv(os.path.join(outdir, 'facility_expansion.csv'), index=False)
    df_builds.to_csv(os.path.join(outdir, 'zip_builds.csv'), index=False)

In [25]:
import matplotlib.pyplot as plt

def _minmax(df):
    return (df - df.min()) / (df.max() - df.min() + 1e-9)

def _plot_heatmap(matrix, row_labels, col_labels, title, out_path):
    plt.figure(figsize=(max(6, 0.4*len(col_labels)), max(6, 0.3*len(row_labels))))
    im = plt.imshow(matrix, aspect='auto', interpolation='nearest')
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.title(title)
    plt.xticks(ticks=np.arange(len(col_labels)), labels=col_labels, rotation=45, ha='right')
    plt.yticks(ticks=np.arange(len(row_labels)), labels=row_labels)
    plt.tight_layout()
    plt.savefig(out_path, dpi=200)
    plt.close()

def make_heatmaps(outdir, df_zip, df_fac, top_k_zip=40, top_k_fac=40,
                  zip_metric='gap_012', fac_metric='expand_x'):
    # ---------- ZIP-level ----------
    if df_zip is not None and not df_zip.empty and zip_metric in df_zip.columns:
        cols_zip = [c for c in [
            'base_012','target_012','after_012',
            'base_05','target_05','after_05',
            'exp_012_zip','exp_05_zip','build_scale_zip','build_05',
            
        ] if c in df_zip.columns]

        top_zip = (df_zip
                   .replace([np.inf, -np.inf], np.nan)
                   .fillna(0)
                   .sort_values(zip_metric, ascending=False)
                   .head(top_k_zip)
                  ).copy()

        data_zip = top_zip[cols_zip].copy()
        data_zip_norm = _minmax(data_zip)

        row_labels = top_zip['zipcode'].astype(str).tolist()
        _plot_heatmap(
            data_zip_norm.to_numpy(),
            row_labels=row_labels,
            col_labels=cols_zip,
            title=f'ZIP Heatmap (Top {top_k_zip} by {zip_metric})',
            out_path=os.path.join(outdir, f'heatmap_zip_top{top_k_zip}_{zip_metric}.png')
        )

    # Facility level output
    if df_fac is not None and not df_fac.empty and fac_metric in df_fac.columns:
        df_fac = df_fac.replace([np.inf, -np.inf], np.nan).fillna(0)

        cols_fac = [c for c in [
            'cap_now','cap_add_max','expand_x','expand_ratio'
        ] if c in df_fac.columns]

        top_fac = (df_fac
                   .sort_values(fac_metric, ascending=False)
                   .head(top_k_fac)
                  ).copy()

        if 'facility_id' in top_fac.columns and 'zipcode' in top_fac.columns:
            row_labels = (top_fac['zipcode'].astype(str) + '#' + top_fac['facility_id'].astype(str)).tolist()
        else:
            row_labels = [str(i) for i in top_fac.index.tolist()]

        data_fac = top_fac[cols_fac].copy()
        data_fac_norm = _minmax(data_fac)

        _plot_heatmap(
            data_fac_norm.to_numpy(),
            row_labels=row_labels,
            col_labels=cols_fac,
            title=f'Facility Heatmap (Top {top_k_fac} by {fac_metric})',
            out_path=os.path.join(outdir, f'heatmap_fac_top{top_k_fac}_{fac_metric}.png')
        )

In [26]:
def solve_statewide(mipgap=1e-4, time_limit=None, verbose=False, outdir='outputs_problem1'):
    m = _make_model(mipgap=mipgap, time_limit=time_limit, verbose=verbose)

    fac_idx, fac_zip, cap_now, cap_add_max, x_f, y_f = add_facility_expansion_vars(m, facility)
    nS, nM, nL, b05, e05, thr_zip = add_zip_level_vars(m, Z_all, is_hd, max_builds, NEW_TYPES)
    facs_in_zip = add_linking_constraints(m, Z_all, fac_idx, fac_zip, x_f, e05, b05, nS, nM, nL, NEW_TYPES)

    add_coverage_constraints(
        m, Z_all, thr_zip, pop_012, pop_05, zip_now_012, zip_now_05,
        facs_in_zip, x_f, nS, nM, nL, e05, b05, NEW_TYPES
    )

    set_objective(
        m, fac_idx, cap_now, x_f, y_f,
        Z_all, e05, b05, nS, nM, nL, NEW_TYPES, EQUIP_COST_U5
    )

    m.optimize()

    status_ok_set = {GRB.OPTIMAL, GRB.TIME_LIMIT}
    status, obj, df_fac = optimize_and_collect_fac(
        m, status_ok_set, fac_idx, fac_zip, cap_now, cap_add_max, x_f, y_f
    )

    df_zip, df_builds = build_zip_summary(
        df_fac, Z_all, thr_zip, zip_now_012, zip_now_05,
        pop_012, pop_05, NEW_TYPES, EQUIP_COST_U5,
        nS, nM, nL, b05, e05, obj
    )

    write_outputs(outdir, status, obj, df_zip, df_fac, df_builds)
    make_heatmaps(outdir, df_zip, df_fac, top_k_zip=40, top_k_fac=40,
              zip_metric='gap_012', fac_metric='expand_x')

    return {
        'status': status,
        'objective': obj,
        'df_zip': df_zip,
        'df_fac': df_fac,
        'df_builds': df_builds
    }


In [27]:
if __name__ == '__main__':
    res = solve_statewide(mipgap=1e-4, time_limit=None, verbose=False, outdir='outputs_problem1')
    print("Status:", res['status'])
    print("Objective:", res['objective'])

Set parameter Username
Set parameter LicenseID to value 2722020
Academic license - for non-commercial use only - expires 2026-10-14
Set parameter MIPGap to value 0.0001
Status: optimal
Objective: 177124266.92659706
